In [1]:
#dependencies
import pandas as pd
state_data = "Resources/StateData.csv"
geocodes_data = "Resources/Geocodes_USA_with_Counties.csv"
CHR_data = "Resources/CHR_Data.csv"
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sem
from scipy.stats import linregress
import numpy as np
from sklearn import datasets

In [2]:
#create state df
state_df = pd.read_csv(state_data)
red_state_df = state_df.loc[:, ["State", "Code"]]
red_state_df.head()

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [3]:
#create geocodes df
geocodes_df = pd.read_csv(geocodes_data)
red_geocodes_df = geocodes_df.loc[:, ["county", "state","zip","latitude","longitude"]]
# red_geocodes_df["zip"] = map(lambda x: x.zfill(5), red_geocodes_df["zip"])
# zero_zip = zip_str.zfill(5)
# red_geocodes_df["zip"] = red_geocodes_df["zip"].astype(str)
red_geocodes_df = red_geocodes_df.rename(columns={"county":"County", "state":"Code", "latitude":"Lat","longitude":"Lon"})
red_geocodes_df.head()
# zero_zip
# red_geocodes_df["zip"].head()
len(red_geocodes_df)

42522

In [4]:
red_geocodes_df = red_geocodes_df.drop_duplicates(subset=("County", "Code"), keep = "first")
len(red_geocodes_df)

3231

In [5]:
#create data frame
#https://thispointer.com/pandas-skip-rows-while-reading-csv-file-to-a-dataframe-using-read_csv-in-python/
CHR_df = pd.read_csv(CHR_data, skiprows = 1)
CHR_df.head()

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,95% CI - Low,95% CI - High,Quartile,YPLL Rate (AIAN),YPLL Rate (AIAN) 95% CI - Low,...,% Drive Alone (Hispanic) 95% CI - Low,% Drive Alone (Hispanic) 95% CI - High,% Drive Alone (White),% Drive Alone (White) 95% CI - Low,% Drive Alone (White) 95% CI - High,# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.18,95% CI - High.18,Quartile.34
0,1000,Alabama,NaN,81791.0,9943.0,9841.0,10045.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2051445.0,34.0,34.0,35.0,NaN
1,1001,Alabama,Autauga,791.0,8129.0,7283.0,8974.0,1.0,NaN,NaN,...,NaN,NaN,84.0,80.0,89.0,24428.0,40.0,36.0,43.0,3.0
2,1003,Alabama,Baldwin,2967.0,7354.0,6919.0,7790.0,1.0,NaN,NaN,...,56.0,81.0,83.0,81.0,85.0,91420.0,42.0,39.0,44.0,3.0
3,1005,Alabama,Barbour,472.0,10254.0,8782.0,11725.0,2.0,NaN,NaN,...,NaN,NaN,87.0,85.0,90.0,8538.0,32.0,27.0,37.0,2.0
4,1007,Alabama,Bibb,471.0,11978.0,10344.0,13611.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7946.0,50.0,42.0,58.0,4.0


In [6]:
red_CHR_df = CHR_df.loc[:, ["FIPS", "State", "County",
                                    "Years of Potential Life Lost Rate", "% Fair or Poor Health", "Average Number of Physically Unhealthy Days", 
                                    "Average Number of Mentally Unhealthy Days", "% Adults with Obesity","Food Environment Index", "% With Access to Exercise Opportunities",
                                    "% Uninsured", "Primary Care Physicians Ratio","Preventable Hospitalization Rate","Population", "% Unemployed",
                                    "Income Ratio", "Social Association Rate", "% Smokers", "% Excessive Drinking", "High School Graduation Rate"]]
red_CHR_df

,FIPS,State,County,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Adults with Obesity,Food Environment Index,% With Access to Exercise Opportunities,% Uninsured,Primary Care Physicians Ratio,Preventable Hospitalization Rate,Population,% Unemployed,Income Ratio,Social Association Rate,% Smokers,% Excessive Drinking,High School Graduation Rate
0,1000,Alabama,NaN,9943.0,22,4.9,4.9,36,5.8,61.0,11.0,1543:01:00,5805.0,1232042,3.9,5.3,12.4,21,14,89.0
1,1001,Alabama,Autauga,8129.0,21,4.7,4.7,33,7.2,69.0,9.0,2220:01:00,7108.0,14438,3.6,5.2,12.1,18,15,90.0
2,1003,Alabama,Baldwin,7354.0,18,4.2,4.3,31,8.0,74.0,11.0,1372:01:00,4041.0,48703,3.6,4.4,10.2,17,18,86.0
3,1005,Alabama,Barbour,10254.0,30,5.4,5.2,42,5.6,53.0,12.0,3159:01:00,6209.0,6779,5.2,5.7,7.5,22,13,81.0
4,1007,Alabama,Bibb,11978.0,19,4.6,4.6,38,7.8,16.0,10.0,2061:01:00,5961.0,5970,4.0,4.4,8.4,19,16,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56037,Wyoming,Sweetwater,7832.0,15,3.4,3.6,30,7.7,90.0,14.0,2721:01:00,2854.0,12573,4.2,4.0,10.3,18,23,81.0
3189,56039,Wyoming,Teton,2731.0,12,3.0,3.2,12,8.2,100.0,16.0,862:01:00,2130.0,7972,3.0,3.8,16.3,15,26,88.0
3190,56041,Wyoming,Uinta,7331.0,16,3.6,3.7,36,7.4,84.0,14.0,2277:01:00,3442.0,5152,4.4,4.4,2.9,17,19,85.0
3191,56043,Wyoming,Washakie,6586.0,16,3.6,3.7,29,8.3,83.0,17.0,2016:01:00,3979.0,1767,4.2,3.3,16.1,17,17,85.0


In [7]:
clean_df = red_CHR_df.dropna()
len(clean_df)

2709

In [9]:
# # pick top and bottom counties and create study dataframe

# n_cities=20

# top_n = clean_df.nlargest(n_cities, ['Preventable Hospitalization Rate'])
# bottom_n = clean_df.nsmallest(n_cities, ['Preventable Hospitalization Rate'])
# study_df = pd.concat([top_n,bottom_n])

# study_df = study_df.reset_index(drop=True)
# study_df.head()

In [8]:
study_state_data_df = pd.merge(clean_df,red_state_df,on="State")
# study_state_data_df.head()
len(study_state_data_df)

2709

In [9]:
#add geocode data
study_state_geodata_df = pd.merge(study_state_data_df,red_geocodes_df,on=["Code","County"], how = 'left')
study_state_geodata_df

,FIPS,State,County,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Adults with Obesity,Food Environment Index,% With Access to Exercise Opportunities,...,% Unemployed,Income Ratio,Social Association Rate,% Smokers,% Excessive Drinking,High School Graduation Rate,Code,zip,Lat,Lon
0,1001,Alabama,Autauga,8129.0,21,4.7,4.7,33,7.2,69.0,...,3.6,5.2,12.1,18,15,90.0,AL,36003.0,32.43,-86.65
1,1003,Alabama,Baldwin,7354.0,18,4.2,4.3,31,8.0,74.0,...,3.6,4.4,10.2,17,18,86.0,AL,36507.0,30.88,-87.77
2,1005,Alabama,Barbour,10254.0,30,5.4,5.2,42,5.6,53.0,...,5.2,5.7,7.5,22,13,81.0,AL,36016.0,31.88,-85.45
3,1007,Alabama,Bibb,11978.0,19,4.6,4.6,38,7.8,16.0,...,4.0,4.4,8.4,19,16,84.0,AL,35034.0,32.94,-87.17
4,1009,Alabama,Blount,11335.0,22,4.9,4.9,34,8.4,16.0,...,3.5,4.4,8.4,19,14,93.0,AL,35013.0,33.90,-86.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,56037,Wyoming,Sweetwater,7832.0,15,3.4,3.6,30,7.7,90.0,...,4.2,4.0,10.3,18,23,81.0,WY,82322.0,42.23,-107.55
2705,56039,Wyoming,Teton,2731.0,12,3.0,3.2,12,8.2,100.0,...,3.0,3.8,16.3,15,26,88.0,WY,83001.0,43.47,-110.76
2706,56041,Wyoming,Uinta,7331.0,16,3.6,3.7,36,7.4,84.0,...,4.4,4.4,2.9,17,19,85.0,WY,82930.0,41.25,-110.96
2707,56043,Wyoming,Washakie,6586.0,16,3.6,3.7,29,8.3,83.0,...,4.2,3.3,16.1,17,17,85.0,WY,82401.0,44.00,-107.95


In [10]:
study_state_geodata2_df = study_state_geodata_df.drop_duplicates(subset=("FIPS"), keep = "first")
study_state_geodata2_df.head()
# len(study_state_geodata2_df)

,FIPS,State,County,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Adults with Obesity,Food Environment Index,% With Access to Exercise Opportunities,...,% Unemployed,Income Ratio,Social Association Rate,% Smokers,% Excessive Drinking,High School Graduation Rate,Code,zip,Lat,Lon
0,1001,Alabama,Autauga,8129.0,21,4.7,4.7,33,7.2,69.0,...,3.6,5.2,12.1,18,15,90.0,AL,36003.0,32.43,-86.65
1,1003,Alabama,Baldwin,7354.0,18,4.2,4.3,31,8.0,74.0,...,3.6,4.4,10.2,17,18,86.0,AL,36507.0,30.88,-87.77
2,1005,Alabama,Barbour,10254.0,30,5.4,5.2,42,5.6,53.0,...,5.2,5.7,7.5,22,13,81.0,AL,36016.0,31.88,-85.45
3,1007,Alabama,Bibb,11978.0,19,4.6,4.6,38,7.8,16.0,...,4.0,4.4,8.4,19,16,84.0,AL,35034.0,32.94,-87.17
4,1009,Alabama,Blount,11335.0,22,4.9,4.9,34,8.4,16.0,...,3.5,4.4,8.4,19,14,93.0,AL,35013.0,33.90,-86.51


In [11]:
study_state_geodata2_df = study_state_geodata2_df.dropna()
len(study_state_geodata2_df)

2615

In [14]:
study_state_geodata2_df.to_csv("Resources/FullData.csv")